In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torchvision import transforms
from PIL import Image
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
from timm.models import create_model

# Function to load image data
def load_image_data(index, base_station_folder='output_images'):
    image_path = f'{base_station_folder}/BS{base_station + 1}/Image_Row_{index + 1}_BS{base_station + 1}.png'

    image = Image.open(image_path).convert('RGB')

    # Convert to tensor
    image = torch.tensor(np.array(image), dtype=torch.float32).permute(2, 0, 1) / 255.0

    return image

# Loading data
Out_set_file = loadmat('DLCB_dataset/O1_60/DLCB_output.mat')
Out_set = Out_set_file['DL_output'].astype(np.float32)

num_user_tot = 20  # Change this if needed
DL_size_ratio = 0.2
DL_size = int(num_user_tot * DL_size_ratio)

np.random.seed(2016)
num_train = int(DL_size * 0.8)
num_test = int(num_user_tot * 0.2)
num_epochs = 50

train_index, test_index = train_test_split(range(num_user_tot), test_size=num_test, random_state=2016)
Out_train, Out_test = Out_set[train_index], Out_set[test_index]

# Number of base stations
num_base_stations = 4
training_history = [[] for _ in range(num_base_stations)]
# Divide output data into segments for each base station
output_segments_train = torch.chunk(torch.tensor(Out_train), num_base_stations, dim=1)

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self, input_channels, output_size):
        super(CNNModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Adjusted input size for the fully connected layer
        last_conv_dim = 256 * 8 * 8  # Update this based on the size after the last convolution
        self.fc = nn.Linear(last_conv_dim, output_size)

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Define the MobileViT model
class MobileViTModel(nn.Module):
    def __init__(self, input_channels, output_size, pretrained=True):
        super(MobileViTModel, self).__init__()

        # Load MobileViT model (freeze pre-trained weights by default)
        self.mobilevit = create_model('vit_base_patch16_224', pretrained=pretrained)
        for param in self.mobilevit.parameters():
            param.requires_grad = False  # Freeze pre-trained weights

        # Access feature extraction layers based on timm version (replace if necessary)
        self.features = self.mobilevit.blocks  # Assuming 'blocks' contains feature extraction layers

        # Determine output dimension of MobileViT (might vary depending on the variant)
        self.mobilevit_out_features = self.mobilevit.head.in_features  # Assuming 'head' leads to final output

        # Add a fully connected layer to reshape the output
        self.fc = nn.Linear(self.mobilevit_out_features * 16, output_size)  # Assuming 16 patches

    def forward(self, x):
        # Reshape input tensor to match the expected shape for the MobileViT model
        x = x.view(x.size(0), -1, 768)  # Reshape to [batch_size, num_patches, 768]

        # Feature extraction using MobileViT
        x = self.features(x)

        # Flatten the output tensor before passing through the fully connected layer
        x = x.view(x.size(0), -1)

        # Apply the fully connected layer
        x = self.fc(x)

        return x




class HybridModel(nn.Module):
    def __init__(self, cnn_model, mobilevit_model, hidden_layer_size, output_size, final_output_size):
        super(HybridModel, self).__init__()
        self.cnn_model = cnn_model
        self.mobilevit_model = mobilevit_model

        # Define fully connected layers
        self.fc1 = nn.Linear(hidden_layer_size, output_size)
        self.fc2 = nn.Linear(output_size, final_output_size)

    def forward(self, x):
        # Forward pass through CNN model
        cnn_output = self.cnn_model(x)
        # Forward pass through MobileViT model
        mobilevit_output = self.mobilevit_model(x)

        # Combine predictions (you can use any method for aggregation, e.g., averaging)
        combined_output = (cnn_output + mobilevit_output) / 2  # Simple average
        
        # Get the shape of combined_output dynamically
        combined_output_shape = combined_output.shape[-1]

        # Pass combined_output through additional layers
        x = nn.ReLU()(self.fc1(combined_output))
        x = nn.ReLU()(self.fc2(x))

        return x



# Training loop
base_station_models = []

for base_station in range(num_base_stations):
    # Instantiate CNN model
    cnn_model = CNNModel(input_channels=3, output_size=512)  # Assuming RGB images
    # Instantiate MobileViT model
    mobilevit_model = MobileViTModel(input_channels=3, output_size=512)  # Assuming RGB images

    # Instantiate the Hybrid model
    hidden_layer_size = 512
    output_size = 256
    final_output_size = 256  # Assuming 10 classes for classification

    # Instantiate the HybridModel
    hybrid_model = HybridModel(cnn_model, mobilevit_model, hidden_layer_size, output_size, final_output_size)

    optimizer = torch.optim.Adam(hybrid_model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    train_dataset = TensorDataset(torch.stack([load_image_data(index, base_station_folder='output_images') for index in train_index]), output_segments_train[base_station])
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    


    total_batches = len(train_dataloader)

    for epoch in range(num_epochs):
        print(f"Training Base Station {base_station + 1}, Epoch {epoch + 1}")
        epoch_losses = []
        for batch_idx, (batch_inputs, batch_targets) in enumerate(train_dataloader):
            optimizer.zero_grad()
            predictions = hybrid_model(batch_inputs)
            loss = criterion(predictions, batch_targets)
            loss.backward()
            optimizer.step()
            epoch_losses.append(loss.item())
            # Print progress
            #percent_complete = (batch_idx + 1) / total_batches * 100
            #sys.stdout.write(f"\rProgress: [{int(percent_complete)}%]")
            #sys.stdout.flush()

        #print()  # Move to the next line after completing an epoch
        training_history[base_station].append(np.mean(epoch_losses))

    base_station_models.append(hybrid_model)

# Evaluation on the test set

input_segments_test = torch.stack([load_image_data(index, base_station_folder='output_images') for index in test_index])
output_segments_test = torch.tensor(Out_test)

def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch_inputs, batch_targets in dataloader:
            predictions = model(batch_inputs)
            
            # Ensure predictions and targets have the same shape
            batch_size = batch_targets.size(0)
            predictions = predictions.view(batch_size, -1)  # Flatten predictions
            batch_targets = batch_targets.view(batch_size, -1)  # Flatten targets
            
            # Compute the loss
            loss = criterion(predictions, batch_targets)
            
            total_loss += loss.item()
    return total_loss / len(dataloader)




base_station_losses = []
# Evaluate each base station model
for base_station, model in enumerate(base_station_models):
    test_dataset = TensorDataset(input_segments_test, output_segments_test[:, base_station])  # Adjust the dimensions
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    test_loss = evaluate_model(model, test_dataloader)
    base_station_losses.append(test_loss)
    print(f"Base Station {base_station + 1} Test Loss: {test_loss}")

# Calculate the mean of MSE for all base stations
mean_mse = np.mean(base_station_losses)
print(f"Mean MSE across all Base Stations: {mean_mse}")

# Save the mean MSE to a CSV file
mean_mse_df = pd.DataFrame({"Mean_MSE": [mean_mse]})
mean_mse_df.to_csv("mean_mse_results.csv", index=False)

# Plotting the training history
plt.figure(figsize=(10, 6))
for base_station, history in enumerate(training_history):
    plt.plot(range(1, num_epochs + 1), history, label=f"Base Station {base_station + 1}")

plt.title('Training History of Hybrid Model')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.grid(True)

# Set both the lower and upper y-axis limits
plt.ylim(0.0005, plt.ylim()[1])

# Save the figure as a PDF
plt.savefig("training_history_plot.pdf", bbox_inches='tight')

# Show the plot
plt.show()

############ Attack the RF Beamforming Codeword Prediction Model ##########
print("Attack")

def fgsm_attack(model, input_data, target_data, epsilon):
    model.eval()
    input_data.requires_grad = True

    # Forward pass
    output = model(input_data)

    # Calculate loss only for the specific base station
    batch_size = output.size(0)
    target_slice = target_data[:, 4*base_station:4*(base_station+1)]  # Slice target data for the specific base station

    # Resize output tensor to match the shape of the target slice
    output_resized = output[:, :target_slice.size(1)]

    loss = nn.MSELoss()(output_resized, target_slice)

    # Backward pass
    model.zero_grad()
    loss.backward()

    # FGSM attack
    input_data_grad = input_data.grad.data
    perturbed_input = input_data + epsilon * torch.sign(input_data_grad)

    return perturbed_input

# Choose an epsilon value for the attack
# Choose an epsilon value for the attack
epsilon = 0.5

# Generate adversarial examples for the training set
adversarial_examples_train = []
for base_station, model in enumerate(base_station_models):
    input_data = torch.stack([load_image_data(index, base_station_folder='output_images') for index in train_index]).clone().detach().requires_grad_(True)
    target_data = output_segments_train[base_station].clone().detach()  # Target data for this base station


    perturbed_input = fgsm_attack(model, input_data, target_data, epsilon)
    
    adversarial_examples_train.append(perturbed_input)

# Evaluate the model on the adversarial examples generated from the training set
adversarial_losses_train = []
for base_station, model in enumerate(base_station_models):
    adversarial_dataset_train = TensorDataset(adversarial_examples_train[base_station], output_segments_train[base_station])
    adversarial_dataloader_train = DataLoader(adversarial_dataset_train, batch_size=32, shuffle=False)
    adversarial_loss_train = evaluate_model(model, adversarial_dataloader_train)
    adversarial_losses_train.append(adversarial_loss_train)
    print(f"Base Station {base_station + 1} Adversarial Train Loss: {adversarial_loss_train}")

# Calculate the mean of MSE for all base stations on adversarial examples generated from the training set
mean_adversarial_mse_train = np.mean(adversarial_losses_train)
print(f"Mean Adversarial Train MSE across all Base Stations: {mean_adversarial_mse_train}")

# Save the mean Adversarial Train MSE to a CSV file
mean_adversarial_mse_train_df = pd.DataFrame({"Mean_Adversarial_Train_MSE": [mean_adversarial_mse_train]})
mean_adversarial_mse_train_df.to_csv("mean_adversarial_train_mse_results.csv", index=False)




# Combine original training data with adversarial examples
original_inputs = torch.stack([load_image_data(index, base_station_folder='output_images') for index in train_index])
original_targets = output_segments_train[base_station]
adversarial_inputs = torch.cat(adversarial_examples_train, dim=0)
adversarial_targets = output_segments_train[base_station]  # Assuming adversarial targets are the same as original targets

# Resize adversarial inputs to match the size of original inputs
adversarial_inputs = adversarial_inputs[:original_inputs.size(0)]  



# Concatenate inputs and targets
combined_inputs = torch.cat((original_inputs, adversarial_inputs), dim=0)
combined_targets = torch.cat((original_targets, adversarial_targets), dim=0)

# Create combined dataset and dataloader
combined_train_dataset = TensorDataset(combined_inputs, combined_targets)
train_dataloader = DataLoader(combined_train_dataset, batch_size=32, shuffle=True)

# Training loop
#base_station_models = []
total_batches = len(train_dataloader)
for epoch in range(num_epochs):
    print(f"Training Base Station {base_station + 1}, Epoch {epoch + 1}")
    epoch_losses = []
    for batch_idx, (batch_inputs, batch_targets) in enumerate(train_dataloader):
        optimizer.zero_grad()
        predictions = hybrid_model(batch_inputs)
        loss = criterion(predictions, batch_targets)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    training_history[base_station].append(np.mean(epoch_losses))

base_station_models.append(hybrid_model)

# Evaluate the model on the adversarial examples generated from the training set
adversarial_losses_train = []
for base_station, model in enumerate(base_station_models):
    adversarial_dataset_train = TensorDataset(input_segments_test, output_segments_test[:, base_station])  # Adjust the dimensions
    adversarial_dataloader_train = DataLoader(test_dataset, batch_size=32, shuffle=False)
    adversarial_loss_train = evaluate_model(model, test_dataloader)
    adversarial_losses_train.append(adversarial_loss_train)
    print(f"Base Station {base_station + 1} Adversarial Train Loss: {adversarial_loss_train}")

# Calculate the mean of MSE for all base stations on adversarial examples generated from the training set
mean_adversarial_mse_train = np.mean(adversarial_losses_train)
print(f"Mean Adversarial Train MSE across all Base Stations: {mean_adversarial_mse_train}")

# Save the mean Adversarial Train MSE to a CSV file
mean_adversarial_mse_train_df = pd.DataFrame({"Mean_Adversarial_Train_MSE": [mean_adversarial_mse_train]})
mean_adversarial_mse_train_df.to_csv("mean_adversarial_train_mse_results.csv", index=False)

 

 



Training Base Station 1, Epoch 1
Training Base Station 1, Epoch 2
Training Base Station 1, Epoch 3
Training Base Station 1, Epoch 4
Training Base Station 1, Epoch 5
Training Base Station 1, Epoch 6
Training Base Station 1, Epoch 7
Training Base Station 1, Epoch 8
Training Base Station 1, Epoch 9
Training Base Station 1, Epoch 10
Training Base Station 1, Epoch 11
Training Base Station 1, Epoch 12
Training Base Station 1, Epoch 13
Training Base Station 1, Epoch 14
Training Base Station 1, Epoch 15
Training Base Station 1, Epoch 16
Training Base Station 1, Epoch 17
Training Base Station 1, Epoch 18
Training Base Station 1, Epoch 19
Training Base Station 1, Epoch 20
Training Base Station 1, Epoch 21
Training Base Station 1, Epoch 22
Training Base Station 1, Epoch 23
Training Base Station 1, Epoch 24
Training Base Station 1, Epoch 25
Training Base Station 1, Epoch 26
Training Base Station 1, Epoch 27
Training Base Station 1, Epoch 28
Training Base Station 1, Epoch 29
Training Base Station 1